In [ ]:
import os

# Mount File System for Working on Colaboratory

# Running or importing .py Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Change working directory

In [ ]:
!ls

In [ ]:
!ls 'drive/My Drive/pytorch-EverybodyDanceNow/notebook'

In [ ]:
import os
os.chdir('drive/My Drive/pytorch-EverybodyDanceNow/notebook')

In [ ]:
import os
import numpy as np
import torch
import time
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.autograd import Variable
from pathlib import Path

%matplotlib inline

In [ ]:
pix2pixhd_dir = Path('../src/pix2pixHD/')

import sys
sys.path.append(str(pix2pixhd_dir))

print(str(pix2pixhd_dir))

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

# Download and extract video

In [ ]:
import cv2
from pathlib import Path

In [ ]:
save_dir = Path('../data/source/')

print(save_dir)

# comment (save_dir.mkdir() ) if path exists
#save_dir.mkdir(True)
img_dir = save_dir.joinpath('images')
#img_dir.mkdir()

print(img_dir)

# Get Frames from Video

In [ ]:
print(str(save_dir.joinpath('file_name.mp4')))
cap = cv2.VideoCapture( str(save_dir.joinpath('file_name.mp4')))
print(cap.isOpened())

i = 0
while(cap.isOpened()):
    flag, frame = cap.read()
    
    if flag == False:
        break 
    cv2.imwrite(str(img_dir.joinpath(f'img_{i:05d}.png')), frame)
    i += 1
print("Number of frames:", i)

# Pose estimation (OpenPose)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

%matplotlib inline

In [ ]:
openpose_dir = Path('../src/pytorch_Realtime_Multi-Person_Pose_Estimation/')

import sys
sys.path.append(str(openpose_dir))
sys.path.append('../src/utils')



In [ ]:
import sys
#!{sys.executable} -m pip install scikit-image

In [ ]:
# openpose
from network.rtpose_vgg import get_model
from evaluate.coco_eval import get_multiplier, get_outputs

# utils
from openpose_utils import remove_noise, get_pose

In [ ]:
weight_name = openpose_dir.joinpath('network/weight/pose_model.pth')

model = get_model('vgg19')     
model.load_state_dict(torch.load(weight_name))
model = torch.nn.DataParallel(model).cuda()
model.float()
model.eval()


## check

In [ ]:
print(img_dir)

print(len(os.listdir(img_dir)))

img_path = sorted(img_dir.iterdir())[300]
img = cv2.imread(str(img_path))
shape_dst = np.min(img.shape[:2])
# offset
oh = (img.shape[0] - shape_dst) // 2
ow = (img.shape[1] - shape_dst) // 2

img = img[oh:oh+shape_dst, ow:ow+shape_dst]
img = cv2.resize(img, (512, 512))
          
plt.imshow(img[:,:,[2, 1, 0]]) # BGR -> RGB

In [ ]:
multiplier = get_multiplier(img)
with torch.no_grad():
    paf, heatmap = get_outputs(multiplier, img, model, 'rtpose')
    
r_heatmap = np.array([remove_noise(ht)
                      for ht in heatmap.transpose(2, 0, 1)[:-1]])\
                     .transpose(1, 2, 0)
heatmap[:, :, :-1] = r_heatmap
param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
label = get_pose(param, heatmap, paf)

plt.imshow(label)

## make label images for pix2pix

In [ ]:
test_img_dir = save_dir.joinpath('test_img')
test_img_dir.mkdir(exist_ok=True)
test_label_dir = save_dir.joinpath('test_label')
test_label_dir.mkdir(exist_ok=True)

for idx in tqdm(range(1882)):
    img_path = img_dir.joinpath(f'img_{idx:05d}.png')
    img = cv2.imread(str(img_path))
    shape_dst = np.min(img.shape[:2])
    oh = (img.shape[0] - shape_dst) // 2
    ow = (img.shape[1] - shape_dst) // 2

    img = img[oh:oh+shape_dst, ow:ow+shape_dst]
    img = cv2.resize(img, (512, 512))
    multiplier = get_multiplier(img)
    with torch.no_grad(): 
        paf, heatmap = get_outputs(multiplier, img, model, 'rtpose')
    r_heatmap = np.array([remove_noise(ht)
                      for ht in heatmap.transpose(2, 0, 1)[:-1]])\
                     .transpose(1, 2, 0)
    heatmap[:, :, :-1] = r_heatmap
    param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
    label = get_pose(param, heatmap, paf)
    cv2.imwrite(str(test_img_dir.joinpath(f'img_{idx:05d}.png')), img)
    cv2.imwrite(str(test_label_dir.joinpath(f'label_{idx:05d}.png')), label)
    
torch.cuda.empty_cache()

In [ ]:
len(os.listdir(str(img_dir)))